In [1]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results 

In [2]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ········


In [3]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages([
    ("system", (
    "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )), 
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")    
])

In [5]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [6]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})

tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"x": 10.0, "y": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6b0338d8-33e6-4d36-9754-2169b414a957-0', tool_calls=[{'name': 'add', 'args': {'x': 10.0, 'y': 10.0}, 'id': 'fd7f9891-4ccc-4620-94ca-9198d4a17cbf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 276, 'output_tokens': 82, 'total_tokens': 358, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 62}})

In [7]:
name2tool = {tool.name: tool.func for tool in tools}

name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

In [8]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec_content

20.0

In [9]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content = f"The { tool_call.tool_calls[0]['name'] }  tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke(
    {
        "input": "What is 10 + 10",
        "chat_history": [],
        "agent_scratchpad": [tool_call, tool_exec]
    }
)

out

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"x": 10.0, "y": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7e845403-e5b7-48bf-9ed5-953ec7ed540d-0', tool_calls=[{'name': 'add', 'args': {'x': 10.0, 'y': 10.0}, 'id': '9c5e595c-d638-4856-9ee3-a4c1992599b7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 318, 'output_tokens': 116, 'total_tokens': 434, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 96}})

In [10]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"x": 10.0, "y": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--cfdb62ca-e8e2-46d1-acd7-e8d9d93a6d48-0', tool_calls=[{'name': 'add', 'args': {'x': 10.0, 'y': 10.0}, 'id': '99708116-5fd5-45a0-8da1-46527c81320f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 276, 'output_tokens': 81, 'total_tokens': 357, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 61}})

In [11]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='10 + 10 = 20.0', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--93eebdfa-1a89-4364-a2cd-763b32097cdd-0', usage_metadata={'input_tokens': 317, 'output_tokens': 12, 'total_tokens': 329, 'input_token_details': {'cache_read': 0}})

In [12]:
@tool
def final_answer(answer: str, tools_used:list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}
    

In [13]:
tools = [final_answer, add, subtract, multiply, exponentiate]

name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [14]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10.0, 'y': 10.0},
  'id': 'f46e7a84-f463-4bfc-93f7-ea2d44998c04',
  'type': 'tool_call'}]

In [15]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={'function_call': {'name': 'final_answer', 'arguments': '{"answer": "10 + 10 is 20.", "tools_used": ["add"]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--bc72ec75-46c9-405c-910d-44c4819a478b-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': '10 + 10 is 20.', 'tools_used': ['add']}, 'id': '1af1eceb-201c-4dec-b44d-04cc8a73d2e3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 421, 'output_tokens': 96, 'total_tokens': 517, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 65}})

In [16]:
out.tool_calls[0]["args"]

{'answer': '10 + 10 is 20.', 'tools_used': ['add']}

In [17]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage

In [64]:
import json

class CustomAgentExecutor:
    chat_history: list[BaseMessage]
    
    
    def __init__(self):
        self.chat_history = [] 
        
        self.agent: RunnableSerializable= (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])                
            }
            | prompt
            | llm.bind_tools(tools, tool_choice = "any")
        )
        
    
    def invoke(self, input: str, max_iterations:int = 3) -> dict:
        count = 0
        agent_scratchpad = []
        self.max_iterations = max_iterations
        
        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            
            agent_scratchpad.append(tool_call)
            
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            
            tool_exec = ToolMessage(
                content = tool_out,
                tool_call_id = tool_call_id
            )
            
            agent_scratchpad.append(tool_exec)
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            
            if tool_name == "final_answer":
                break
        
        if isinstance(tool_out, dict) and "answer" in tool_out:
            final_answer = tool_out["answer"]
            self.chat_history.extend([
                HumanMessage(content=input),
                AIMessage(content=final_answer)
            ])
            return json.dumps(tool_out)

        print("Need more iterations to get the answer")
        
        return json.dumps({})

In [65]:
agent_executor_custom = CustomAgentExecutor()



In [66]:
agent_executor_custom.invoke(input = "What is 10 + 10")

0: add({'x': 10.0, 'y': 10.0})
1: final_answer({'answer': '10 + 10 = 20', 'tools_used': ['add']})


'{"answer": "10 + 10 = 20", "tools_used": ["add"]}'

In [67]:
agent_executor_custom.invoke(input = "What is nine plus 10, minus 4 * 2 to the power of 3", max_iterations = 5)

0: exponentiate({'x': 2.0, 'y': 3.0})
1: multiply({'x': 4.0, 'y': 8.0})
2: add({'x': 9.0, 'y': 10.0})
3: subtract({'x': 32.0, 'y': 19.0})
4: final_answer({'answer': 'The result is -13.', 'tools_used': ['exponentiate', 'multiply', 'add', 'subtract']})


'{"answer": "The result is -13.", "tools_used": ["exponentiate", "multiply", "add", "subtract"]}'